In [1]:
#파일없을때 src 공란, json을 변경, src 경로 수정

#파일없을때 src 공란, json을 변경

import json
import os
import shutil
import re
from pathlib import Path

# ==================== 설정 영역 (여기를 수정하세요) ====================

# 이미지 원본 경로들 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\daumeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

# 이미지 복사본 저장 폴더
OUTPUT_IMAGE_FOLDER = r"C:\Users\LEEJUHWAN\Downloads\board_club_img"

# 처리할 JSON 파일
JSON_FILE = r"board_posts.json"

# src에서 제거할 접두사 (이 부분을 제거하고 상대경로로 변경)
REMOVE_PREFIX = r"C:\Users\LEEJUHWAN\Downloads"

# ======================================================================


def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None


def process_single_post(data, post_index):
    """하나의 post 데이터 처리"""
    
    # sourceId에서 문서ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return data  # 원본 그대로 반환
    
    doc_id = source_id.lstrip('_')
    folder_name = f"brd{doc_id}"
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{post_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return data  # 원본 그대로 반환
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('content', '')
    if not html_content:
        print(f"   ⚠️ content 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
            
            # src 경로 변경 (상대경로로)
            new_src = str(new_file).replace('\\', '/')
            if REMOVE_PREFIX:
                prefix_with_slash = REMOVE_PREFIX.replace('\\', '/')
                if new_src.startswith(prefix_with_slash):
                    new_src = new_src[len(prefix_with_slash):].lstrip('/')
            
            new_html = new_html.replace(old_src, new_src, 1)
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # src를 빈 문자열로 변경
            new_html = new_html.replace(old_src, '', 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['content'] = new_html
    return data


def process_json_file(json_file_path):
    """JSON 파일 처리"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # 1. JSON 파일 읽기
    print(f"📂 파일 읽는 중: {json_file_path}")
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    posts = data.get('posts', [])
    total_count = len(posts)
    
    print(f"✅ 총 {total_count}개의 posts 발견\n")
    
    # 2. 각 post 처리
    processed_posts = []
    success_count = 0
    
    for idx, post in enumerate(posts, 1):
        try:
            result = process_single_post(post, idx)
            processed_posts.append(result)
            if result != post:  # 변경사항이 있으면
                success_count += 1
        except Exception as e:
            print(f"❌ [Post {idx}] 처리 오류: {e}")
            processed_posts.append(post)  # 오류 시 원본 유지
            continue
    
    # 3. 수정된 데이터 저장
    output_file = json_file_path.replace('.json', '_imgConvert.json')
    
    output_data = {"posts": processed_posts}
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n{'='*60}")
    print(f"✅ 처리 완료!")
    print(f"   - 처리된 posts: {success_count}/{total_count}개")
    print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
    print(f"   - 수정된 파일: {output_file}")
    print(f"{'='*60}")


# 실행
if __name__ == "__main__":
    process_json_file(JSON_FILE)

📂 파일 읽는 중: board_posts.json
📂 검색할 이미지 경로: 4개

✅ 총 33개의 posts 발견


📄 [1] 문서ID: 3249916
   원본: C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor\brd3249916
   복사: C:\Users\LEEJUHWAN\Downloads\board_club_img\brd3249916
   ✅ 이미지 3개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 1.jpg
      ✅ 2 → 2.jpg
   📊 복사 완료: 3/3개

📄 [2] 문서ID: 2800116
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor\brd2800116
   복사: C:\Users\LEEJUHWAN\Downloads\board_club_img\brd2800116
   ✅ 이미지 12개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 1.jpg
      ✅ 2 → 2.jpg
      ✅ 3 → 3.jpg
      ✅ 4 → 4.jpg
      ✅ 5 → 5.jpg
      ✅ 6 → 6.jpg
      ✅ 7 → 7.jpg
      ✅ 8 → 8.jpg
      ✅ 9 → 9.jpg
      ✅ 10 → 10.jpg
      ✅ 11 → 11.jpg
   📊 복사 완료: 12/12개

📄 [3] 문서ID: 2658426
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor\brd2658426
   복사: C:\Users\LEEJUHWAN\Downloads\board_club_img\brd2658426
   ✅ 이미지 2개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 

In [2]:
#파일없을때 src 공란, json을 변경

import json
import os
import shutil
import re
from pathlib import Path

# 설정 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\daumeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

OUTPUT_IMAGE_FOLDER = r"C:\Users\LEEJUHWAN\Downloads\board_qna_img"
JSON_FILE = r"board_posts.json"  # JSON 파일명

def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None

def process_single_post(data, post_index):
    """하나의 post 데이터 처리"""
    
    # sourceId에서 문서ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return data  # 원본 그대로 반환
    
    doc_id = source_id.lstrip('_')
    folder_name = f"brd{doc_id}"
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{post_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return data  # 원본 그대로 반환
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('content', '')
    if not html_content:
        print(f"   ⚠️ content 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
            # src 경로 변경
            new_src = str(new_file).replace('\\', '/')
            new_html = new_html.replace(old_src, new_src, 1)
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # src를 빈 문자열로 변경
            new_html = new_html.replace(old_src, '', 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['content'] = new_html
    return data

def process_json_file(json_file_path):
    """JSON 파일 처리"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # 1. JSON 파일 읽기
    print(f"📂 파일 읽는 중: {json_file_path}")
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    posts = data.get('posts', [])
    total_count = len(posts)
    
    print(f"✅ 총 {total_count}개의 posts 발견\n")
    
    # 2. 각 post 처리
    processed_posts = []
    success_count = 0
    
    for idx, post in enumerate(posts, 1):
        try:
            result = process_single_post(post, idx)
            processed_posts.append(result)
            if result != post:  # 변경사항이 있으면
                success_count += 1
        except Exception as e:
            print(f"❌ [Post {idx}] 처리 오류: {e}")
            processed_posts.append(post)  # 오류 시 원본 유지
            continue
    
    # 3. 수정된 데이터 저장
    output_file = json_file_path.replace('.json', '_imgConvert.json')
    
    output_data = {"posts": processed_posts}
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n{'='*60}")
    print(f"✅ 처리 완료!")
    print(f"   - 처리된 posts: {success_count}/{total_count}개")
    print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
    print(f"   - 수정된 파일: {output_file}")
    print(f"{'='*60}")

# 실행
if __name__ == "__main__":
    process_json_file(JSON_FILE)

📂 파일 읽는 중: board_posts.json
📂 검색할 이미지 경로: 4개

✅ 총 79개의 posts 발견

⚠️ [1] 폴더 없음: brd3792176 (모든 경로 검색 완료)
⚠️ [2] 폴더 없음: brd2911241 (모든 경로 검색 완료)
⚠️ [3] 폴더 없음: brd2614054 (모든 경로 검색 완료)

📄 [4] 문서ID: 2608297
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor\brd2608297
   복사: C:\Users\LEEJUHWAN\Downloads\board_qna_img\brd2608297
   ✅ 이미지 1개 발견
      ✅ 0 → 0.jpg
   📊 복사 완료: 1/1개
⚠️ [5] 폴더 없음: brd2609354 (모든 경로 검색 완료)
⚠️ [6] 폴더 없음: brd2541843 (모든 경로 검색 완료)
⚠️ [7] 폴더 없음: brd2479030 (모든 경로 검색 완료)
⚠️ [8] 폴더 없음: brd2420619 (모든 경로 검색 완료)
⚠️ [9] 폴더 없음: brd2390131 (모든 경로 검색 완료)
⚠️ [10] 폴더 없음: brd2273106 (모든 경로 검색 완료)
⚠️ [11] 폴더 없음: brd2273103 (모든 경로 검색 완료)
⚠️ [12] 폴더 없음: brd2031164 (모든 경로 검색 완료)
⚠️ [13] 폴더 없음: brd1925837 (모든 경로 검색 완료)
⚠️ [14] 폴더 없음: brd972092 (모든 경로 검색 완료)
⚠️ [15] 폴더 없음: brd969534 (모든 경로 검색 완료)
⚠️ [16] 폴더 없음: brd969225 (모든 경로 검색 완료)
⚠️ [17] 폴더 없음: brd865557 (모든 경로 검색 완료)
⚠️ [18] 폴더 없음: brd865500 (모든 경로 검색 완료)
⚠️ [19] 폴더 없음: brd865498 (모든 경로 검색 완

In [ ]:
# 파일이 없을때 img src를 공란으로 (테스트 아직 안해봄. txt(cmds)를 변경하는 코드

import json
import os
import shutil
import re
from pathlib import Path

# 설정 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

OUTPUT_IMAGE_FOLDER = r"C:\Users\LEEJUHWAN\Downloads\converted_images"
ADDPOST_FILE = r"converted_posts.txt"

def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None

def process_single_post(data, post_index):
    """하나의 addPost 데이터 처리"""
    
    # sourceId에서 문서ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return None
    
    doc_id = source_id.lstrip('_')
    folder_name = f"brd{doc_id}"
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{post_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return None
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('content', '')
    if not html_content:
        print(f"   ⚠️ content 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
            # src 경로 변경
            new_src = str(new_file).replace('\\', '/')
            new_html = new_html.replace(old_src, new_src, 1)
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name} - src 공란 처리")
            # src를 빈 문자열로 변경
            new_html = new_html.replace(old_src, '', 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['content'] = new_html
    return data

def process_addpost_file(addpost_file_path):
    """전체 파일 처리 - 줄 단위로 읽기"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # 1. 파일 읽기 (줄 단위)
    print(f"📂 파일 읽는 중: {addpost_file_path}")
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    processed_data = []
    success_count = 0
    total_count = 0
    
    with open(addpost_file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            
            # 빈 줄 건너뛰기
            if not line:
                continue
            
            # addPost로 시작하는지 확인
            if not line.startswith('addPost'):
                print(f"⚠️ [줄 {line_num}] 'addPost'로 시작하지 않음")
                continue
            
            total_count += 1
            
            # 'addPost' 제거하고 JSON 파싱
            json_str = line[7:]  # 'addPost' (7글자) 제거
            
            try:
                data = json.loads(json_str)
                result = process_single_post(data, total_count)
                if result:
                    processed_data.append(result)
                    success_count += 1
            except json.JSONDecodeError as e:
                print(f"❌ [줄 {line_num}] JSON 파싱 오류: {e}")
                print(f"   처음 100자: {json_str[:100]}")
                continue
            except Exception as e:
                print(f"❌ [줄 {line_num}] 처리 오류: {e}")
                continue
    
    print(f"\n✅ 총 {total_count}개의 addPost 발견")
    
    # 2. 수정된 데이터 저장
    if processed_data:
        output_file = addpost_file_path.replace('.txt', '_modified.txt')
        with open(output_file, 'w', encoding='utf-8') as f:
            for data in processed_data:
                f.write('addPost' + json.dumps(data, ensure_ascii=False) + '\n')
        
        print(f"\n{'='*60}")
        print(f"✅ 처리 완료!")
        print(f"   - 성공: {success_count}/{total_count}개")
        print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
        print(f"   - 수정된 파일: {output_file}")
        print(f"{'='*60}")
    else:
        print("\n❌ 처리된 데이터가 없습니다.")

# 실행
if __name__ == "__main__":
    process_addpost_file(ADDPOST_FILE)

In [12]:
import json
import os
import shutil
import re
from pathlib import Path

# 설정 - 여러 경로 추가 가능
BASE_IMAGE_PATHS = [
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor",
    # 더 추가하고 싶으면 여기에 계속 추가
]

OUTPUT_IMAGE_FOLDER = r"C:\Users\LEEJUHWAN\Downloads\converted_images"
ADDPOST_FILE = r"converted_posts.txt"

def find_image_folder(folder_name):
    """여러 경로에서 이미지 폴더 찾기"""
    for base_path in BASE_IMAGE_PATHS:
        image_folder = Path(base_path) / folder_name
        if image_folder.exists():
            return image_folder
    return None

def process_single_post(data, post_index):
    """하나의 addPost 데이터 처리"""
    
    # sourceId에서 문서ID 추출
    source_id = data.get('sourceId', '')
    if not source_id:
        print(f"⚠️ [{post_index}] sourceId 없음, 건너뜀")
        return None
    
    doc_id = source_id.lstrip('_')
    folder_name = f"brd{doc_id}"
    
    # 여러 경로에서 이미지 폴더 찾기
    image_folder = find_image_folder(folder_name)
    
    if not image_folder:
        print(f"⚠️ [{post_index}] 폴더 없음: {folder_name} (모든 경로 검색 완료)")
        return None
    
    # 복사본 저장할 문서별 폴더 생성
    output_doc_folder = Path(OUTPUT_IMAGE_FOLDER) / folder_name
    output_doc_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"\n📄 [{post_index}] 문서ID: {doc_id}")
    print(f"   원본: {image_folder}")
    print(f"   복사: {output_doc_folder}")
    
    # HTML에서 img 태그 찾기
    html_content = data.get('content', '')
    if not html_content:
        print(f"   ⚠️ content 없음")
        return data
    
    img_pattern = r'<img[^>]+src="([^"]+)"[^>]*>'
    img_matches = list(re.finditer(img_pattern, html_content))
    
    if not img_matches:
        print(f"   ℹ️ 이미지 없음")
        return data
    
    print(f"   ✅ 이미지 {len(img_matches)}개 발견")
    
    # 파일 복사 & src 경로 변경
    new_html = html_content
    copied_count = 0
    
    for idx, match in enumerate(img_matches):
        old_src = match.group(1)
        
        # 원본 파일 (0, 1, 2...)
        old_file = image_folder / str(idx)
        
        # 복사본 파일 (.jpg 확장자 추가)
        new_file = output_doc_folder / f"{idx}.jpg"
        
        # 파일 복사
        if old_file.exists():
            shutil.copy2(old_file, new_file)
            copied_count += 1
            print(f"      ✅ {old_file.name} → {new_file.name}")
        else:
            print(f"      ⚠️ 파일 없음: {old_file.name}")
            continue
        
        # src 경로 변경
        new_src = str(new_file).replace('\\', '/')
        new_html = new_html.replace(old_src, new_src, 1)
    
    print(f"   📊 복사 완료: {copied_count}/{len(img_matches)}개")
    
    # 수정된 HTML 적용
    data['content'] = new_html
    return data

def process_addpost_file(addpost_file_path):
    """전체 파일 처리 - 줄 단위로 읽기"""
    
    # 출력 폴더 생성
    os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
    
    # 1. 파일 읽기 (줄 단위)
    print(f"📂 파일 읽는 중: {addpost_file_path}")
    print(f"📂 검색할 이미지 경로: {len(BASE_IMAGE_PATHS)}개\n")
    
    processed_data = []
    success_count = 0
    total_count = 0
    
    with open(addpost_file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            
            # 빈 줄 건너뛰기
            if not line:
                continue
            
            # addPost로 시작하는지 확인
            if not line.startswith('addPost'):
                print(f"⚠️ [줄 {line_num}] 'addPost'로 시작하지 않음")
                continue
            
            total_count += 1
            
            # 'addPost' 제거하고 JSON 파싱
            json_str = line[7:]  # 'addPost' (7글자) 제거
            
            try:
                data = json.loads(json_str)
                result = process_single_post(data, total_count)
                if result:
                    processed_data.append(result)
                    success_count += 1
            except json.JSONDecodeError as e:
                print(f"❌ [줄 {line_num}] JSON 파싱 오류: {e}")
                print(f"   처음 100자: {json_str[:100]}")
                continue
            except Exception as e:
                print(f"❌ [줄 {line_num}] 처리 오류: {e}")
                continue
    
    print(f"\n✅ 총 {total_count}개의 addPost 발견")
    
    # 2. 수정된 데이터 저장
    if processed_data:
        output_file = addpost_file_path.replace('.txt', '_modified.txt')
        with open(output_file, 'w', encoding='utf-8') as f:
            for data in processed_data:
                f.write('addPost' + json.dumps(data, ensure_ascii=False) + '\n')
        
        print(f"\n{'='*60}")
        print(f"✅ 처리 완료!")
        print(f"   - 성공: {success_count}/{total_count}개")
        print(f"   - 복사된 이미지 위치: {OUTPUT_IMAGE_FOLDER}")
        print(f"   - 수정된 파일: {output_file}")
        print(f"{'='*60}")
    else:
        print("\n❌ 처리된 데이터가 없습니다.")

# 실행
if __name__ == "__main__":
    process_addpost_file(ADDPOST_FILE)

📂 파일 읽는 중: converted_posts.txt
📂 검색할 이미지 경로: 2개


📄 [1] 문서ID: 3249916
   원본: C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\resource\image\namoeditor\brd3249916
   복사: C:\Users\LEEJUHWAN\Downloads\converted_images\brd3249916
   ✅ 이미지 3개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 1.jpg
      ✅ 2 → 2.jpg
   📊 복사 완료: 3/3개

📄 [2] 문서ID: 2800116
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor\brd2800116
   복사: C:\Users\LEEJUHWAN\Downloads\converted_images\brd2800116
   ✅ 이미지 12개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 1.jpg
      ✅ 2 → 2.jpg
      ✅ 3 → 3.jpg
      ✅ 4 → 4.jpg
      ✅ 5 → 5.jpg
      ✅ 6 → 6.jpg
      ✅ 7 → 7.jpg
      ✅ 8 → 8.jpg
      ✅ 9 → 9.jpg
      ✅ 10 → 10.jpg
      ✅ 11 → 11.jpg
   📊 복사 완료: 12/12개

📄 [3] 문서ID: 2658426
   원본: C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\resource\image\daumeditor\brd2658426
   복사: C:\Users\LEEJUHWAN\Downloads\converted_images\brd2658426
   ✅ 이미지 2개 발견
      ✅ 0 → 0.jpg
      ✅ 1 → 1.jpg
   📊